# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [20]:
# import the requests library (1 line)

import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)

root_url = 'https://country-leaders.onrender.com'

# assign the /status endpoint to another variable called status_url (1 line)

status_url = '/status'

# query the /status endpoint using the get() method and store it in the req variable (1 line)

req = requests.get(f"{root_url}{status_url}")

# check the status_code using a condition and print appropriate messages (4 lines)

status_code = req.status_code
print(f"Status code for request is {status_code}")
if status_code >= 200 and status_code < 300 :
    print("Success!")
    print(req.text)
elif status_code >= 400 and status_code < 500:
    print("Client error! Verify request!")
elif status_code >= 500:
    print("Server error")



Status code for request is 200
Success!
"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [21]:
# Set the countries_url variable (1 line)

countries_url = '/countries'

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

header = {'accept':'application/json'}
req = requests.get(f"{root_url}{countries_url}, headers = header")

# Get the JSON content and store it in the countries variable (1 line)

countries = req.json()

# display the request's status code and the countries variable (1 line)

print(f"Status code for request is {req.status_code}\nRetrieved following data: {countries}")

Status code for request is 404
Retrieved following data: {'detail': 'Not Found'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [22]:
# Set the cookie_url variable (1 line)

cookie_url = '/cookie'

# Query the enpoint, set the cookies variable and display it (2 lines)

req = requests.get(f"{root_url}{cookie_url}")
cookies = req.cookies


In [18]:
def get_cookie():
    req = requests.get(f"{root_url}{cookie_url}")
    return req.cookies

Try to query the countries endpoint using the cookie, save the output and print it.

In [15]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

countries = requests.get(f"{root_url}{countries_url}", cookies=cookies).json()

# display the countries variable (1 line)

print(countries)

['us', 'be', 'ma', 'ru', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [23]:
# Set the leaders_url variable (1 line)

leaders_url = '/leaders'

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

leaders = requests.get(f"{root_url}{leaders_url}", cookies=cookies).json()

# display the leaders variable (1 line)

print(leaders)


{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [26]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)

params = {'country':countries[1], 'accept':'application/json'}
cookie = get_cookie()

leaders = requests.get(f"{root_url}{leaders_url}", params= params, cookies= cookie).json()

# display the leaders variable (1 line)

leaders


[{'id': 'Q12978',
  'first_name': 'Guy',
  'last_name': 'Verhofstadt',
  'birth_date': '1953-04-11',
  'death_date': None,
  'place_of_birth': 'Dendermonde',
  'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt',
  'start_mandate': '1999-07-12',
  'end_mandate': '2008-03-20'},
 {'id': 'Q12981',
  'first_name': 'Yves',
  'last_name': 'Leterme',
  'birth_date': '1960-10-06',
  'death_date': None,
  'place_of_birth': 'Wervik',
  'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme',
  'start_mandate': '2009-11-25',
  'end_mandate': '2011-12-06'},
 {'id': 'Q12983',
  'first_name': 'Herman',
  'last_name': 'None',
  'birth_date': '1947-10-31',
  'death_date': None,
  'place_of_birth': 'Etterbeek',
  'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy',
  'start_mandate': '2008-12-30',
  'end_mandate': '2009-11-25'},
 {'id': 'Q14989',
  'first_name': 'Léon',
  'last_name': 'Delacroix',
  'birth_date': '1867-12-27',
  'death_date': '1929-10-15',
  'place_of_

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [27]:
# 4 lines
leaders_per_country = {}
cookie = get_cookie()
for country in countries:
    params = {'country':country, 'accept':'application/json'}
    leaders_per_country[country] = requests.get(f"{root_url}{leaders_url}", params= params, cookies= cookie).json()


In [29]:
# or 1 line
cookie = get_cookie()
leaders_per_country = {country:requests.get(f"{root_url}{leaders_url}", params= {'country':country, 'accept':'application/json'}, cookies= cookie).json() for country in countries}

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [16]:
# < 15 lines

def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = '/cookie'
    countries_url = '/countries'
    leaders_url = '/leaders'

    cookie = get_cookie()
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookie).json()

    leaders_per_country = dict()
    for country in countries:
        params = {'country':country, 'accept':'application/json'}
        leaders_per_country[country] = requests.get(f"{root_url}{leaders_url}", params= params, cookies= cookie).json()

    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [28]:
# 2 lines

leaders_per_country = get_leaders()
leaders_per_country


{'us': [{'id': 'Q23',
   'first_name': 'George',
   'last_name': 'Washington',
   'birth_date': '1732-02-22',
   'death_date': '1799-12-14',
   'place_of_birth': 'Westmoreland County',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington',
   'start_mandate': '1789-04-30',
   'end_mandate': '1797-03-04'},
  {'id': 'Q76',
   'first_name': 'Barack',
   'last_name': 'Obama',
   'birth_date': '1961-08-04',
   'death_date': None,
   'place_of_birth': 'Kapiolani Medical Center for Women and Children',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama',
   'start_mandate': '2009-01-20',
   'end_mandate': '2017-01-20'},
  {'id': 'Q91',
   'first_name': 'Abraham',
   'last_name': 'Lincoln',
   'birth_date': '1809-02-12',
   'death_date': '1865-04-15',
   'place_of_birth': 'Sinking Spring Farm',
   'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln',
   'start_mandate': '1861-03-04',
   'end_mandate': '1865-04-15'},
  {'id': 'Q207',
   'first_name': 'Ge

In [26]:
total_length = len(leaders_per_country['us']) + len(leaders_per_country['fr']) + len(leaders_per_country['ru']) + len(leaders_per_country['ma']) + len(leaders_per_country['be'])
total_length

136

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [229]:
# 3 lines
url = leaders_per_country['us'][21]['wikipedia_url']
leader_wiki = requests.get(url)
leader_wiki.text


'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Martin Van Buren - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 v

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [230]:
# 3 lines
from bs4 import BeautifulSoup

soup = BeautifulSoup(leader_wiki.content, "html")
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Martin Van Buren - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-c

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [231]:
# 2 lines
paragraphs = soup.find_all("p")
paragraphs

[<p class="mw-empty-elt">
 </p>,
 <p><b>Martin Van Buren</b> (<span class="rt-commentedText nowrap"><span class="IPA nopopups noexcerpt" lang="en-fonipa"><a href="/wiki/Help:IPA/English" title="Help:IPA/English">/<span style="border-bottom:1px dotted"><span title="'v' in 'vie'">v</span><span title="/æ/: 'a' in 'bad'">æ</span><span title="'n' in 'nigh'">n</span></span><span class="wrap"> </span><span style="border-bottom:1px dotted"><span title="/ˈ/: primary stress follows">ˈ</span><span title="'b' in 'buy'">b</span><span title="/jʊər/: 'ure' in 'cure'">jʊər</span><span title="/ən/: 'on' in 'button'">ən</span></span>/</a></span></span> <a href="/wiki/Help:Pronunciation_respelling_key" title="Help:Pronunciation respelling key"><i title="English pronunciation respelling">van <span style="font-size:90%">BURE</span>-ən</i></a>; <a href="/wiki/Dutch_language" title="Dutch language">Dutch</a>: <i lang="nl">Maarten van Buren</i> <span class="IPA nowrap" lang="nl-Latn-fonipa"><a href="/wiki/Hel

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [202]:
# <10 lines
import re

regex = r'\d+'

for paragraph in paragraphs:

    if re.findall(regex, paragraph.text):
        print("this paragraph matches")
        first_paragraph = paragraph.text
        break
    else:
        print("no match")

print(first_paragraph)

this paragraph matches
Gaston François Marie burggraaf Eyskens (Lier, 1 april 1905 – Leuven, 3 januari 1988) was drie maal christendemocratisch eerste minister van België in de periode tussen 1949 en 1973. Hij was de eerste Belgische regeringsleider die geboren was in de 20e eeuw.



In [232]:
birth_year = '1782-12-05'[:4]

for paragraph in paragraphs:
    if re.findall(birth_year, paragraph.text):
        print("this paragraph matches")
        first_paragraph = paragraph.text
        break
    else:
        print("no match")

print(first_paragraph)

no match
this paragraph matches
Martin Van Buren (/væn ˈbjʊərən/ van BURE-ən; Dutch: Maarten van Buren [ˈmaːrtə(ɱ) vɑm ˈbyːrə(n)] ⓘ; December 5, 1782 – July 24, 1862) was the eighth president of the United States, serving from 1837 to 1841. A primary founder of the Democratic Party, he served as New York's attorney general and U.S. senator, then briefly as the ninth governor of New York before joining Andrew Jackson's administration as the tenth United States secretary of state, minister to Great Britain, and ultimately the eighth vice president from 1833 to 1837, after being elected on Jackson's ticket in 1832. Van Buren won the presidency in 1836 against divided Whig opponents. Van Buren lost re-election in 1840, and failed to win the Democratic nomination in 1844. Later in his life, Van Buren emerged as an elder statesman and an anti-slavery leader who led the Free Soil Party ticket in the 1848 presidential election.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url: str, leader_birth_year: str) -> str:

    print(wikipedia_url)
    leader_wiki = requests.get(wikipedia_url).content
    soup = BeautifulSoup(leader_wiki, "html")
    paragraphs = soup.find_all("p")

    for paragraph in paragraphs:
        if re.findall(leader_birth_year, paragraph.text):
            return paragraph.text

In [16]:
# Test: 3 lines


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [243]:
# 3 lines

regex_reference = r'\[[0-9a-z]{1,2}\]'
regex_phonetic_french = r'\(.*?Écouter.?\)'
regex_phonetic_dutch = r'\(.*?uitspraak.*?\)'
regex_phonetic_english = r'\(.*?;\s?.*;\s?'

string = "Martin Van Buren (/væn ˈbjʊərən/ van BURE-ən; Dutch: Maarten van Buren [ˈmaːrtə(ɱ) vɑm ˈbyːrə(n)] ⓘ; December 5, 1782 – July 24, 1862) was the eighth president of the United States, serving from 1837 to 1841. A primary founder of the Democratic Party, he served as New York's attorney general and U.S. senator, then briefly as the ninth governor of New York before joining Andrew Jackson's administration as the tenth United States secretary of state, minister to Great Britain, and ultimately the eighth vice president from 1833 to 1837, after being elected on Jackson's ticket in 1832. Van Buren won the presidency in 1836 against divided Whig opponents. Van Buren lost re-election in 1840, and failed to win the Democratic nomination in 1844. Later in his life, Van Buren emerged as an elder statesman and an anti-slavery leader who led the Free Soil Party ticket in the 1848 presidential election."

re.sub(regex_phonetic_english, '(', string)
#re.sub(regex_reference, '', string)


"Martin Van Buren (December 5, 1782 – July 24, 1862) was the eighth president of the United States, serving from 1837 to 1841. A primary founder of the Democratic Party, he served as New York's attorney general and U.S. senator, then briefly as the ninth governor of New York before joining Andrew Jackson's administration as the tenth United States secretary of state, minister to Great Britain, and ultimately the eighth vice president from 1833 to 1837, after being elected on Jackson's ticket in 1832. Van Buren won the presidency in 1836 against divided Whig opponents. Van Buren lost re-election in 1840, and failed to win the Democratic nomination in 1844. Later in his life, Van Buren emerged as an elder statesman and an anti-slavery leader who led the Free Soil Party ticket in the 1848 presidential election."

Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [251]:
# 10 lines
def get_first_paragraph(wikipedia_url: str, leader_birth_year: str) -> str:

    print(wikipedia_url)
    leader_wiki = requests.get(wikipedia_url).content
    soup = BeautifulSoup(leader_wiki, "html")
    paragraphs = soup.find_all("p")
    regex_reference = r'\[[0-9a-z]{1,2}\]'
    regex_phonetic_french = r'\(.*?Écouter.?\)'
    regex_phonetic_dutch = r'\(.*?uitspraak.*?\)'
    regex_phonetic_english = r'\(.*?;\s?.*;\s?'

    for paragraph in paragraphs:
        if re.findall(leader_birth_year, paragraph.text):
            first_regex = re.sub(regex_reference, '', paragraph.text)
            second_regex = re.sub(regex_phonetic_french, '', first_regex)
            third_regex = re.sub(regex_phonetic_dutch, '', second_regex)
            fourth_regex = re.sub(regex_phonetic_english, '(', third_regex)
            return fourth_regex

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines
# Done above...

## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# < 20 lines

def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = '/cookie'
    countries_url = '/countries'
    leaders_url = '/leaders'

    cookie = get_cookie()
    countries = requests.get(f"{root_url}{countries_url}", cookies=cookie).json()

    leaders_per_country = dict()
    for country in countries:
        params = {'country':country, 'accept':'application/json'}
        if requests.get(f"{root_url}{leaders_url}", params= params, cookies= cookie).status_code == 403:
            cookie = get_cookie()
        leaders = requests.get(f"{root_url}{leaders_url}", params= params, cookies= cookie).json()
        leaders_per_country[country] = leaders
        for index, leader in enumerate(leaders):
            if not isinstance(leader['birth_date'], str):
                continue
            else:
                leader_wiki_url = leader['wikipedia_url']
                leader_birth_year = leader['birth_date'][:4]
                first_paragraph = get_first_paragraph(leader_wiki_url, leader_birth_year)
                leaders_per_country[country][index]['bio'] = first_paragraph
    return leaders_per_country

In [264]:
# Check the output of your function (2 lines)
leaders_per_country = get_leaders()
leaders_per_country

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson


KeyboardInterrupt: 

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# < 25 lines

# Edited inline in previous code block

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)

# Code amended so doesn't crash

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [13]:
# < 20 lines

import re
from bs4 import BeautifulSoup
import requests
from requests import Session

def get_first_paragraph(wikipedia_url: str, leader_birth_year: str, session: Session) -> str:

    print(wikipedia_url)
    leader_wiki = session.get(wikipedia_url).content
    soup = BeautifulSoup(leader_wiki, "html")
    paragraphs = soup.find_all("p")
    regex_reference = r'\[[0-9a-z]{1,2}\]'
    regex_phonetic_french = r'\(.*?Écouter.?\)'
    regex_phonetic_dutch = r'\(.*?uitspraak.*?\)'
    regex_phonetic_english = r'\(.*?;\s?.*;\s?'

    for paragraph in paragraphs:
        if re.findall(leader_birth_year, paragraph.text):
            first_regex = re.sub(regex_reference, '', paragraph.text)
            second_regex = re.sub(regex_phonetic_french, '', first_regex)
            third_regex = re.sub(regex_phonetic_dutch, '', second_regex)
            fourth_regex = re.sub(regex_phonetic_english, '(', third_regex)
            return fourth_regex

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [14]:
# <25 lines
def get_leaders():
    root_url = 'https://country-leaders.onrender.com'
    cookie_url = '/cookie'
    countries_url = '/countries'
    leaders_url = '/leaders'

    session = requests.Session()
    session.get(f"{root_url}{cookie_url}")
    countries = session.get(f"{root_url}{countries_url}").json()

    leaders_per_country = dict()
    for country in countries:
        params = {'country':country, 'accept':'application/json'}
        if session.get(f"{root_url}{leaders_url}", params= params).status_code == 403:
            session.get(f"{root_url}{cookie_url}")
        leaders = session.get(f"{root_url}{leaders_url}", params= params).json()
        leaders_per_country[country] = leaders
        for index, leader in enumerate(leaders):
            if not isinstance(leader['birth_date'], str):
                continue
            else:
                leader_wiki_url = leader['wikipedia_url']
                leader_birth_year = leader['birth_date'][:4]
                first_paragraph = get_first_paragraph(leader_wiki_url, leader_birth_year, session)
                leaders_per_country[country][index]['bio'] = first_paragraph

          
    return leaders_per_country

Test your new functions.



In [15]:
leaders_per_country = get_leaders()
print(leaders_per_country)

https://en.wikipedia.org/wiki/George_Washington
https://en.wikipedia.org/wiki/Barack_Obama
https://en.wikipedia.org/wiki/Abraham_Lincoln
https://en.wikipedia.org/wiki/George_W._Bush
https://en.wikipedia.org/wiki/Bill_Clinton
https://en.wikipedia.org/wiki/Joe_Biden
https://en.wikipedia.org/wiki/Franklin_D._Roosevelt
https://en.wikipedia.org/wiki/Andrew_Johnson
https://en.wikipedia.org/wiki/Gerald_Ford
https://en.wikipedia.org/wiki/Richard_Nixon
https://en.wikipedia.org/wiki/Lyndon_B._Johnson
https://en.wikipedia.org/wiki/John_F._Kennedy
https://en.wikipedia.org/wiki/Dwight_D._Eisenhower
https://en.wikipedia.org/wiki/Ronald_Reagan
https://en.wikipedia.org/wiki/Harry_S._Truman
https://en.wikipedia.org/wiki/James_Madison
https://en.wikipedia.org/wiki/James_Monroe
https://en.wikipedia.org/wiki/John_Quincy_Adams
https://en.wikipedia.org/wiki/Andrew_Jackson
https://en.wikipedia.org/wiki/Martin_Van_Buren
https://en.wikipedia.org/wiki/William_Henry_Harrison
https://en.wikipedia.org/wiki/John_Ty

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [ ]:
# 3 lines
import json



Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [27]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [28]:
# 3 lines


In [29]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!